In [ ]:
#import necessary pacakages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

#read the url file into the pandas object
df = pd.read_excel('/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/Input.xlsx')

#loop throgh each row in the df
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  #write title and text to the file
  file_name = '/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/TitleText' + str(url_id) + '.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article)

can't get response of nan
can't get response of nan
can't get title of blackassign0036
can't get title of blackassign0049


In [ ]:
# Directories
text_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/"
stopwords_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/StopWords"
sentment_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/MasterDictionary"

# load all stop wors from the stopwords directory and store in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
  # Check if it's a file before trying to open
  if os.path.isfile(os.path.join(text_dir, text_file)):
    # Try reading with 'latin-1' encoding
    with open(os.path.join(text_dir,text_file),'r', encoding='latin-1') as f: # Changed encoding to 'latin-1'
      text = f.read()
    #tokenize the given text file
      words = word_tokenize(text)
    # remove the stop words from the tokens
      filtered_text = [word for word in words if word.lower() not in stop_words]
    # add each filtered tokens of each file into a list
      docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

In [ ]:
# Directories
text_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/"
stopwords_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/StopWords"
sentment_dir = "/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/MasterDictionary"

# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

# Use nltk's stopwords instead of the variable
stop_words = set(nltk.corpus.stopwords.words('english'))
def measure(file):
  # Explicitly specify the encoding as 'latin-1'
  with open(os.path.join(text_dir, file),'r', encoding='latin-1') as f:
    text = f.read()
# remove punctuations
    text = re.sub(r'[^\w\s.]','',text)
# split the given text file into sentences
    sentences = text.split('.')
# total number of sentences in a file
    num_sentences = len(sentences)
# total words in the file
    words = [word  for word in text.split() if word.lower() not in stop_words ] # Use stop_words here
    num_words = len(words)

    # Handle the case when num_words is zero to avoid division by zero
    if num_words == 0:
      return 0, 0, 0, 0, 0 # Return default values if there are no words

# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word

    # Handle the case where there are no syllable words to avoid division by zero
    if len(syllable_words) == 0:
      avg_syllable_word_count = 0  # Or any other appropriate default value
    else:
      avg_syllable_word_count = syllable_count / len(syllable_words)

    avg_sentence_len = num_words / num_sentences
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    # Return 5 values to match the unpacking
    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
  # Check if it's a file before trying to process it
  if os.path.isfile(os.path.join(text_dir, file)):
    x,y,z,a,b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_Complex_words.append(y)
    Fog_Index.append(z)
    complex_word_count.append(a)  # Assuming you want to store these values
    avg_syllable_word_count.append(b)


In [ ]:
def cleaned_words(file):
  file_path = os.path.join(text_dir, file)  # Create full file path
  # Check if it's a file before trying to open
  if os.path.isfile(file_path):
    # Explicitly specify the encoding as 'latin-1'
    with open(file_path, 'r', encoding='latin-1') as f:
      text = f.read()
      text = re.sub(r'[^\w\s]', '' , text)
      # Use nltk's stopwords instead of the variable
      words = [word  for word in text.split() if word.lower() not in stop_words]
      length = sum(len(word) for word in words)
      # Handle the case when there are no words to avoid division by zero
      if len(words) == 0:
        average_word_length = 0  # Or any other appropriate default value
      else:
        average_word_length = length / len(words)
    return len(words),average_word_length
  else:
    return 0, 0  # Return default values if it's a directory

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  # Check if it's a file before processing
  if os.path.isfile(os.path.join(text_dir, file)):
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)


def count_personal_pronouns(file):
  file_path = os.path.join(text_dir, file)  # Construct full file path
  # Check if it's a file before trying to open
  if os.path.isfile(file_path):
    # Explicitly specify the encoding as 'latin-1'
    with open(file_path, 'r', encoding='latin-1') as f:
      text = f.read()
      personal_pronouns = ["I", "we", "my", "ours", "us"]
      count = 0
      for pronoun in personal_pronouns:
        count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
    return count
  else:
    return 0  # Return 0 if it's a directory

pp_count = []
for file in os.listdir(text_dir):
  # Check if it's a file before processing
  if os.path.isfile(os.path.join(text_dir, file)):
    x = count_personal_pronouns(file)
    pp_count.append(x)

In [ ]:
output_df = pd.read_excel('/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/Output Data Structure.xlsx')

# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop(output_df[output_df['URL_ID'].isin([44, 57, 144])].index, inplace=True)

# These are the required parameters
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            # avg_sentence_length,  # This was duplicated, removing one
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# Check lengths and adjust if necessary
for i, var in enumerate(variables):
  if len(var) != len(output_df):
    print(f"Warning: Length mismatch for variable at index {i}. Expected {len(output_df)}, got {len(var)}.")
    # Truncate the list to match the DataFrame length:
    var = var[:len(output_df)]
    variables[i] = var # Update the variables list

# write the values to the dataframe
for i, var in enumerate(variables):
  output_df.iloc[:,i+2] = var # Assuming you want to start writing from the third column

#now save the dataframe to the disk
output_df.to_csv('/content/drive/MyDrive/Data_Extraction_and_Text_Analysis_for_Blackcoffer_company.-main/Output_Data.csv')